In [ ]:
!git clone https://github.com/ultralytics/ultralytics

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import time

In [ ]:
%cd /content/ultralytics

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

In [ ]:
import numpy as np

def estimate_distance(x1, y1, x2, y2,focal_length):

    h = y2-y1


    car_height_mm = 1500  # Example vehicle width in millimeters
    truck_height_mm = 3000

    focal_length_pixels = focal_length

    # Estimate the distance based on the width of the bounding box in pixels


    return (car_height_mm * focal_length_pixels) / h

In [ ]:
def draw_bounding_box_with_label(image, x1, y1, x2, y2, label, color, thickness=1):
    # draw bounding box
    cv2.rectangle(image, (x1, y1), (x2, y2), color=color, thickness=thickness)
    # draw a rectangle that contains label
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.7
    label_size, baseline = cv2.getTextSize(label, font, font_scale, thickness=thickness)
    cv2.rectangle(
        image,
        (x1 - int(thickness / 2), y1 - label_size[1]),
        (x1 + label_size[0], y1),
        color,
        cv2.FILLED,
    )
    # draw label
    cv2.putText(
        image, label, (x1, y1), font, font_scale, color=(0, 0, 0)
    )  # black label

In [ ]:
%cd /content
!gdown --fuzzy https://drive.google.com/file/d/1n1qQbeooVDVJ44yW2iG-Mu9iMCYp85z0/view?usp=sharing

In [ ]:
# !pip install moviepy

In [ ]:
# from moviepy.video.io.VideoFileClip import VideoFileClip

# clip = VideoFileClip("/content/highway05.mp4").subclip(0, 10) # Trim video from 0 to 10 seconds

# clip.write_videofile("/content/trimmed_video.mp4")

In [ ]:
image_path = '/content/focal.png'

In [ ]:
results = model.predict(image_path, stream=True,show=True)
for result in results:
  boxes = result.boxes[0].cpu().numpy()
  for box in boxes:                                          # iterate boxes
      r = box.xyxy[0].astype(int)                            # get corner points as int
      print(r)
      x1,y1,x2,y2 = r


In [ ]:
image =cv2.imread(image_path)
for result in results:
  boxes = result.boxes[0].cpu().numpy() # get boxes on cpu in numpy
  for box in boxes: # iterate boxes
   r = box.xyxy[0].astype(int) # get corner points as int
print(r) # print boxes
cv2.rectangle(image, r[:2], r[2:], (0,255,0), 2) # draw boxes on img

In [ ]:
cv2_imshow(image)

In [ ]:
def calc_focal_length(x1,x2,y1,y2):
  height_bbox = y2-y1

  obj_height = 1700

  distance_mm = 3600

  return (height_bbox * distance_mm)/obj_height

In [ ]:
focal_length = calc_focal_length(x1,x2,y1,y2)
print(focal_length)

In [ ]:
%cd /content
!gdown --fuzzy https://drive.google.com/file/d/1KTDY-DwZ9nKHVDfDMA1ZsOse9SHYPch_/view?usp=drive_link

In [ ]:
# Open the video file
video_path = "/content/cam02.avi"
cap = cv2.VideoCapture(video_path)

frame_rate = cap.get(cv2.CAP_PROP_FPS)  # Get the frame rate of the video
print(frame_rate)

In [ ]:
# Define the output video path and properties
output_video_path = "/content/output_video.mp4"
output_video_fps = cap.get(cv2.CAP_PROP_FPS)
output_video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
output_video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create a VideoWriter object to save the processed frames
fourcc = cv2.VideoWriter_fourcc(*"XVID")
output_video = cv2.VideoWriter(output_video_path, fourcc, output_video_fps, (output_video_width, output_video_height))

# Define a dictionary to store the previous distances for each object
previous_distances = {}
previous_frames = {}
frame_rate = cap.get(cv2.CAP_PROP_FPS)  # Get the frame rate of the video

Xe trước tăng tốc: pre_dis < dis -> dis_dap > 0

Xe trước giảm tốc: pre_dis > dis -> dis_gap = p.dis - dis < 0

speed = camera_speed + (distance_gap / time_gap)




In [ ]:
frame_list = []
frame_limit = 5
frame_count = 0

while cap.isOpened():
    success, frame = cap.read()
    if success:
        results = model.track(frame, stream=True, conf=0.25, classes=[2], tracker="botsort.yaml")
        for result in results:
            boxes = result.boxes.cpu().numpy()
            for box in boxes:
                r = box.xyxy[0].astype(int)
                x1, y1, x2, y2 = r

                if box.id is None:
                    box_id = 0
                else:
                    box_id = int(box.id)
                box_id_tuple = tuple([box_id])

                # Estimate distance in meters
                capo_width = 1
                distance = estimate_distance(x1, y1, x2, y2, focal_length) / 1000 - capo_width

                # Check if the object has a previous distance stored
                if box_id_tuple in previous_distances:
                    # Retrieve the previous distance
                    previous_distance = previous_distances[box_id_tuple]

                    # Calculate the distance gap
                    distance_gap = distance - previous_distance

                    # Calculate the time gap in seconds
                    time_gap = 1 / frame_rate  # Assuming the frame rate is constant

                    camera_speed = 50 / 3.6  # m/s
                    # Calculate the speed of infront of car in m/s
                    speed = camera_speed + (distance_gap / time_gap)

                    label = f"car{box_id}: {distance:.1f}m ({distance_gap:.2f}m) speed:{speed:.1f}m/s"
                    color = (0, 255, 0)

                    draw_bounding_box_with_label(frame, x1, y1, x2, y2, label=label, color=color)

                # Save the current distance as the previous distance for the object
                previous_distances[box_id_tuple] = distance

                frame_list.append(frame)

                if len(frame_list) > frame_limit:
                    old_frame = frame_list.pop(0)
                    # Thực hiện các thao tác với frame đầu tiên (old_frame)
                    # ...

                frame_count += 1

                if frame_count == frame_limit:
                    # Tính giá trị trung bình của 5 frame
                    # ...

                    # Cập nhật giá trị mới cho 5 frame
                    # ...

                    # Đặt lại biến đếm frame
                    frame_count = 0

        output_video.write(frame)

        cv2_imshow(frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    else:
        break

cap.release()
output_video.release()
cv2.destroyAllWindows()


In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

input_path = "/content/output_video.mp4"
compress_path = "/content/result.mp4"

os.system(f"ffmpeg -i {input_path} -vcodec libx264 {compress_path}")

mp4 = open(compress_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width 500 controls>
  <source src="%s" type="video/mp4">
</video>
""" %data_url)